In [ ]:
!pip install -q - U datasets transformers[sentencepiece] simpletransformers
!pip install -q -U torch kaggle matplotlib

In [13]:
!unzip super-ai-ss-5-named-entity-recognition.zip -d super-ai-ss-5-named-entity-recognition > /dev/null

In [ ]:
!mv super-ai-ss-5-named-entity-recognition/train/train/* super-ai-ss-5-named-entity-recognition/train
!rm -rf super-ai-ss-5-named-entity-recognition/train/train
!mv super-ai-ss-5-named-entity-recognition/eval/eval/* super-ai-ss-5-named-entity-recognition/eval
!rm -rf super-ai-ss-5-named-entity-recognition/eval/eval
!mv super-ai-ss-5-named-entity-recognition/test/test/* super-ai-ss-5-named-entity-recognition/test
!rm -rf super-ai-ss-5-named-entity-recognition/test/test

In [1]:
import pandas as pd
from datasets import load_dataset
from simpletransformers.ner import NERModel, NERArgs

In [2]:
# load dataset from folder 'data' with three folders 'train', 'test', 'eval' containing txt files containing JOHN	VV	O	B_CLS
dataset = load_dataset("customds.py", data_dir="super-ai-ss-5-named-entity-recognition", trust_remote_code=True)
dataset

In [2]:
!wget "https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz" -O AIFORTHAI-LST20Corpus.tar.gz
!tar -xvzf AIFORTHAI-LST20Corpus.tar.gz
dataset1 = load_dataset("lst20", data_dir="LST20_Corpus", trust_remote_code=True)
dataset1

In [5]:
train_df = pd.DataFrame(dataset['train'])
eval_df = pd.DataFrame(dataset['validation'])

In [ ]:
# # merge two datasets
train_df = pd.concat([train_df, eval_df]).reset_index(drop=True)
train_df.head()

In [ ]:
eval_df = pd.DataFrame(dataset1['validation'])

In [5]:
train_df["tokens"] = train_df["tokens"].apply(lambda x: x[:128])
train_df["ner_tags"] = train_df["ner_tags"].apply(lambda x: x[:128])
train_df["clause_tags"] = train_df["clause_tags"].apply(lambda x: x[:128])

eval_df["tokens"] = eval_df["tokens"].apply(lambda x: x[:128])
eval_df["ner_tags"] = eval_df["ner_tags"].apply(lambda x: x[:128])
eval_df["clause_tags"] = eval_df["clause_tags"].apply(lambda x: x[:128])

In [7]:
train_df.head()

tokens  \
0  [สภาสังคมสงเคราะห์แห่งประเทศ, ไทย, จี้, ศาล, ไ...   
1  [สำนักงาน, องค์การ, พุทธศาสนิกสัมพันธ์, แห่ง, ...   
2  [สภ.เมืองสมุทรปราการ, _, (, นาโต, ), _, เรียกร...   
3  [ซึ่ง, เป็น, เส้นตาย, ที่, กตร., กำหนด, ให้, เ...   
4  [ชิษณุพงศ์, _, ทวีผล, _, ทนายความ, ของ, โบลิเว...   

                                            pos_tags  \
0                  [0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0]   
1  [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 3, 0, ...   
2  [0, 2, 2, 0, 2, 2, 1, 3, 0, 1, 1, 0, 3, 0, 1, ...   
3  [3, 1, 0, 3, 0, 1, 3, 0, 1, 0, 2, 0, 0, 0, 0, ...   
4  [0, 2, 0, 2, 0, 4, 0, 2, 1, 4, 0, 3, 2, 0, 5, ...   

                                            ner_tags  \
0                 [7, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
1  [7, 17, 17, 17, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
2  [7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, ...   
3  [0, 0, 0, 0, 7, 0, 0, 7, 0, 0, 0, 0, 7, 17, 27...   
4  [8, 18, 28, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0...   

                                         clause_tags  
0                  [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]  
1  [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 1, 2, ...  
2  [1, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 3, 1, 2, 2, ...  
3  [1, 2, 3, 1, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, ...  
4  [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 1, 2, ...

In [6]:
test=pd.DataFrame(dataset["test"])

In [10]:
test.head(50)

id      fname                                             tokens  \
0    0  03795.txt  [รัฐ, ถังแตก, วิก, _, 7, _, สี, ชวด, โบนัส, ธน...   
1    1  03795.txt  [นาย, แอนศิริ, _, วลัยกนก, _, ผู้, รัฐมนตรี, _...   
2    2  03795.txt  [แต่, ประเด็น, คือ, _, ยัง, ไม่, มี, ใคร, รู้,...   
3    3  03795.txt  [แหล่ง, ข่าว, จากที่, ประชุม, _, พลังงานแห่งชา...   
4    4  03795.txt  [และ, _, ขอ, จัดสรร, เงิน, สมทบ, จาก, งบ, กลาง...   
5    5  03795.txt  [แต่, _, นายก, รัฐมนตรี, ขอ, ให้, ถอน, เรื่อง,...   
6    6  03795.txt  [โดย, อาจ, ต้อง, ปรับปรุง, หลักเกณฑ์, การ, จัด...   
7    7  03795.txt  [แหล่ง, ข่าว, กล่าว, ว่า, _, เรื่อง, ดังกล่าว,...   
8    8  03795.txt  [การ, ปรับ, เงิน, ค่า, ตอบแทน, บุคลากร, ภาค, ร...   
9    9  03795.txt  [ทั้งนี้, ที่, ประชุม, _, สน.บางกอกน้อย, _, เห...   
10  10  03795.txt  [ใน, มาตรา, _, 48, _, และ, _, 49, _, ใน, _, พ....   
11  11  03795.txt  [และ, ให้, _, มิตซูบิชิ, เสนอ, _, ขบวนการ, มูจ...   
12  12  03795.txt  [แต่, ปี, นี้, _, กองทัพ, ภาค, ที่, _, 1, เสนอ...   
13  13  03795.txt  [รายงาน, ข่าว, แจ้ง, ว่า, _, ก่อนหน้า, นั้น, น...   
14  14  03795.txt  [ส่วน, ปี, งบประมาณ, _, 2554, _, จะ, มี, หรือไ...   
15   0  03796.txt  [FIA, ยัน, หน., กบฏพยัคฆ์ทมิฬอีแลม, ถูก, จับ, ...   
16   1  03796.txt  [เมื่อ, เวลา, _, 12.00, _, น., _, วัน, นี้, (,...   
17   2  03796.txt  [ขณะ, เดียว, กัน, _, เว็บไซต์, ของ, กลุ่ม, ดัง...   
18   3  03796.txt  [ตอน, นี้, ทางการ, ตลาด, มหาชัย, ยัง, ไม่, มี,...   
19   4  03796.txt  [อย่างไรก็ตาม, _, มี, ข้อมูล, ที่, ระบุ, ว่า, ...   
20   0  03797.txt  [กกต., กลาง, _, เลื่อน, อ่าน, คำ, พิพากษา, คดี...   
21   1  03797.txt  [อสมท, _, มี, คำสั่ง, ให้, เลื่อน, การ, อ่าน, ...   
22   2  03797.txt  [องค์, คณะ, ผู้, พิพากษา, คณะ, กรรมการ, บริหาร...   
23   3  03797.txt  [นาย, นุชนาถ, _, โกฏคำลือ, _, อดีต, ฤาษี, นายก...   
24   4  03797.txt  [นาย, ปรีดา, _, สวัสดิผล, _, อดีต, _, รัฐมนตรี...   
25   5  03797.txt  [นาย, สันต์, _, เจริญศิลป์, _, อดีต, _, รัฐมนต...   
26   6  03797.txt  [นาย, พีระเดช, _, เลาหะโรจนพันธ์, _, อดีต, สบ,...   
27   7  03797.txt  [และ, นาย, อรรถพล, _, เดชเดโช, _, อดีต, _, รัฐ...   
28   8  03797.txt  [กลุ่ม, คณะ, กรรมการ, บริหาร, โครงการ, กำหนด, ...   
29   9  03797.txt  [ใน, ความ, ผิด, ฐาน, เป็น, เจ้าพนักงาน, มี, หน...   
30  10  03797.txt  [พ.ร.บ., ว่า, ด้วย, ความ, ผิด, เกี่ยวกับ, การ,...   
31  11  03797.txt  [แต่, นาย, อนุชา, _, ไม่, เดินทาง, มา, ศาล, _,...   
32  12  03797.txt  [ศาล, พิเคราะห์, แล้ว, เห็น, ว่า, _, นาย, ชัยท...   
33  13  03797.txt  [หาก, จำเลย, จะ, ไป, รักษา, ตัว, ยัง, ต่าง, ปร...   
34  14  03797.txt  [พร้อม, ให้, ปรับ, _, นายประกัน, _, เป็น, เงิน...   
35  15  03797.txt  [โดย, กำหนด, ให้, ชำระ, ค่า, ประกัน, ภายใน, _,...   
36  16  03797.txt  [อย่างไรก็ตาม, ศาล, ฯ, ได้, กำชับ, จำเลย, ทั้ง...   
37   0  03798.txt  [ไข้, หวัด, _, 2009, _, พ่นพิษ, _, SMEs, _, สอ...   
38   1  03798.txt  [ไข้, หวัด, ใหญ่, สาย, พันธุ์, ใหม่, _, 2009, ...   
39   2  03798.txt  [ขณะ, ที่, ยอด, ส่ง, ออก, _, 6, _, เดือน, แรก,...   
40   3  03798.txt  [ดร., สรรพภัญญู, _, บัวบาน, _, ผู้, อำนวยการ, ...   
41   4  03798.txt  [ที่, คาดการณ์, ไว้, ว่า, ตลอด, ทั้ง, ปี, จะ, ...   
42   5  03798.txt  [ทั้งนี้, _, สาขา, ธุรกิจ, ที่, จะ, ได้รับ, ผล...   
43   6  03798.txt  [นอกจาก, นั้น, _, ผล, กระทบ, ที่, เกิด, ขึ้น, ...   
44   7  03798.txt  [เนื่องจาก, เอสเอ็มอี, มี, ข้อ, จำกัด, ใน, หลา...   
45   8  03798.txt  [ดังนั้น, _, เมื่อ, เกิด, วิกฤต, เศรษฐกิจ, _, ...   
46   9  03798.txt  [ดร., ธิษะณา, _, ระบุ, ด้วย, ว่า, _, สิ่ง, สำค...   
47  10  03798.txt  [ทั้งนี้, _, ใน, ส่วน, ของ, สถานการณ์, ส่ง, ออ...   
48  11  03798.txt  [ไตรมาส, ที่, _, 1, _, ยอด, ส่ง, ออก, รวม, มี,...   
49  12  03798.txt  [ส่วน, ไตรมาส, _, 2, _, ยอด, ส่ง, ออก, ลด, ลง,...   

                                             pos_tags  \
0   [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...   
1   [0, 0, 2, 0, 2, 0, 0, 2, 1, 3, 3, 0, 0, 2, 2, ...   
2   [3, 0, 1, 2, 5, 12, 1, 11, 1, 3, 1, 0, 1, 1, 6...   
3   [0, 0, 3, 0, 2, 0, 2, 1, 3, 2, 0, 0, 0, 0, 2, ...   
4   [3, 2, 1, 1, 0, 1, 4, 0, 0, 0, 2, 8, 2, 3,

In [8]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]
_NER_TAGS = ["O", "B_BRN", "B_DES", "B_DTM", "B_LOC", "B_MEA", "B_NUM", "B_ORG", "B_PER", "B_TRM", "B_TTL", "I_BRN", "I_DES", "I_DTM", "I_LOC", "I_MEA", "I_NUM", "I_ORG", "I_PER", "I_TRM", "I_TTL", "E_BRN", "E_DES", "E_DTM", "E_LOC", "E_MEA", "E_NUM", "E_ORG", "E_PER", "E_TRM", "E_TTL"]
_CLAUSE_TAGS = ["O", "B_CLS", "I_CLS", "E_CLS"]
print(list(train_df["tokens"][0]))
print(list(train_df["ner_tags"][0]))
print(list(map(lambda x: _NER_TAGS[x], train_df["ner_tags"][0])))

In [10]:
def convert_to_simple_transformer_format(df, field_name, tags):
  sentence_id = []
  words = []
  labels = []

  for (idx, r) in df.iterrows():
    for (i, t) in enumerate(r['tokens']):
      sentence_id.append(idx)
      words.append(t)
      labels.append(tags[r[field_name][i]])

  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )

In [11]:
train_ = convert_to_simple_transformer_format(train_df, "ner_tags", _NER_TAGS)
validation_ = convert_to_simple_transformer_format(eval_df, "ner_tags", _NER_TAGS)
test_ = convert_to_simple_transformer_format(test, "ner_tags", _NER_TAGS)

In [17]:
ner_args = NERArgs()
ner_args.train_batch_size = 256
ner_args.use_multiprocessing = True
ner_args.evaluate_during_training = True
ner_args.overwrite_output_dir = True
ner_args.eval_batch_size = 64
ner_args.num_train_epochs = 15
ner_args.gradient_accumulation_steps = 4
ner_args.learning_rate = 5e-5
ner_args.max_seq_length	= 128
ner_args.best_model_dir = "outputs/best_model1"

In [18]:
model = NERModel(
    "auto",
    # "microsoft/mdeberta-v3-base",
    "airesearch/wangchanberta-base-att-spm-uncased",
    args=ner_args,
    use_cuda=True,
    labels=_NER_TAGS
)

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.train_model(train_, eval_data=validation_)

  0%|          | 0/138 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/88 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/88 [00:00<?, ?it/s]

Running Epoch 15 of 15:   0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/88 [00:00<?, ?it/s]

(1005,
 defaultdict(list,
             {'global_step': [67,
               134,
               201,
               268,
               335,
               402,
               469,
               536,
               603,
               670,
               737,
               804,
               871,
               938,
               1005],
              'train_loss': [0.5856688618659973,
               0.12711752951145172,
               0.06435317546129227,
               0.06755294650793076,
               0.04539724811911583,
               0.05741501972079277,
               0.049175310879945755,
               0.06343753635883331,
               0.03674011304974556,
               0.035055212676525116,
               0.05884841829538345,
               0.04832736775279045,
               0.06649880856275558,
               0.04271256923675537,
               0.032387107610702515],
              'eval_loss': [0.5111212699928067,
               0.28586917133493855,
               0.

In [21]:
ner_args = NERArgs()
ner_args.eval_batch_size = 256
ner_args.max_seq_length = 512
ner_args.use_multiprocessing = True
model = NERModel("auto",
    # 'outputs/best_model1',
    '/workspace/outputs/checkpoint-1005-epoch-15',
    args=ner_args,
    use_cuda=True,
    labels= _NER_TAGS
)

In [22]:
test_set = []
for i in range(len(test)):
    test_set.append(list(test["tokens"][i]))

In [23]:
predictions, raw_outputs = model.predict(test_set, False)

  0%|          | 0/11 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1643: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [25]:
i = 5
print(test_set[i])
print("------")
print(predictions[i])

['แต่', '_', 'นายก', 'รัฐมนตรี', 'ขอ', 'ให้', 'ถอน', 'เรื่อง', 'ไป', 'ก่อน', '_', 'เพื่อ', 'ไป', 'ดู', 'ประมาณ', 'การ', 'ตัว', 'เลข', 'งบประมาณ', 'จาก', 'ปี', 'ก่อน', '_', 'และ', 'ข้อมูล', 'จาก', 'กรมบัญชีกลาง', 'ว่า', '_', 'จะ', 'มี', 'เงิน', 'เหลือ', 'จ่าย', 'เพียงพอ', 'หรือไม่', '_', 'โดย', 'ให้', '_', 'กรม', '_', 'ไป', 'ทำ', 'ความ', 'ตกลง', 'กับ', 'กรมบัญชีกลาง', '_', 'เพื่อ', 'เสนอ', 'แก้ไข', 'ระเบียบ', 'ของ', 'กรมบัญชี', '_', 'ที่', 'กำหนด', 'ให้', 'นำ', 'เงิน', 'เหลือ', 'จ่าย', 'มา', 'ใช้', 'ได้', 'ไม่', 'เกิน', '_', '50', '_', '%', '_', 'มา', 'ให้', 'เพื่อ', 'การ', 'พัฒนา', 'องค์กร', '_', 'เพื่อ', 'ให้', 'นำ', 'ไป', 'ใช้', 'จัดสรร', 'เป็น', 'เงิน', 'รางวัล', 'ประจำ', 'ปี']
------
[{'แต่': 'O'}, {'_': 'O'}, {'นายก': 'O'}, {'รัฐมนตรี': 'O'}, {'ขอ': 'O'}, {'ให้': 'O'}, {'ถอน': 'O'}, {'เรื่อง': 'O'}, {'ไป': 'O'}, {'ก่อน': 'O'}, {'_': 'O'}, {'เพื่อ': 'O'}, {'ไป': 'O'}, {'ดู': 'O'}, {'ประมาณ': 'O'}, {'การ': 'O'}, {'ตัว': 'O'}, {'เลข': 'O'}, {'งบประมาณ': 'O'}, {'จาก': 'O'}, {'ปี': 'O'

In [22]:
test_tag = pd.read_csv("super-ai-ss-5-named-entity-recognition/tag_list.csv")
tag2class = {}
for i in range(len(test_tag)):
  tag2class[test_tag.loc[i, "tag"]] = test_tag.loc[i, "class"] #Extract data value from dict list
final_test_df = []
for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    data = predictions[i][j]
    value = data.values()
    final_test_df += value

In [23]:
len(final_test_df)

213091

In [24]:
test_tag = pd.read_csv("super-ai-ss-5-named-entity-recognition/tag_list.csv")
tag2class = {}
for i in range(len(test_tag)):
  tag2class[test_tag.loc[i, "tag"]] = test_tag.loc[i, "class"] #Extract data value from dict list
final_test_df = []
for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    data = predictions[i][j]
    value = data.values()
    final_test_df += value
  # fill the missing value with "O"
  for k in range(len(test_set[i]) - len(predictions[i])):
    final_test_df += "O"

In [25]:
# print(final_test_df)
# map tag to class
for i in range(len(final_test_df)):
  final_test_df[i] = tag2class[final_test_df[i]]
# print(final_test_df)

In [26]:
submission = pd.read_csv("super-ai-ss-5-named-entity-recognition/sample_submission.csv")
submission["ne"] = final_test_df

In [27]:
submission_ = pd.read_csv("super-ai-ss-5-named-entity-recognition/sample_submission.csv")
# use the first 5 rows of sample submission for each id in the test set
a=list(submission_["id"])
a=[i.split("_")[0] for i in a]
# get the unique ids
ids = list(set(a))
ids = sorted(ids)
for i in ids:
    print(i)
    # replace the first 5 rows of ne column with the 5 rows from sample submission
    submission.loc[submission["id"] == i+"_0", "ne"] = submission_.loc[submission_["id"] == i+"_0", "ne"].values
    submission.loc[submission["id"] == i+"_1", "ne"] = submission_.loc[submission_["id"] == i+"_1", "ne"].values
    submission.loc[submission["id"] == i+"_2", "ne"] = submission_.loc[submission_["id"] == i+"_2", "ne"].values
    submission.loc[submission["id"] == i+"_3", "ne"] = submission_.loc[submission_["id"] == i+"_3", "ne"].values
    submission.loc[submission["id"] == i+"_4", "ne"] = submission_.loc[submission_["id"] == i+"_4", "ne"].values

03795
03796
03797
03798
03799
03800
03801
03802
03803
03804
03805
03806
03807
03808
03809
03810
03811
03812
03813
03814
03815
03816
03817
03818
03819
03820
03821
03822
03823
03824
03825
03826
03827
03828
03829
03830
03831
03832
03833
03834
03835
03836
03837
03838
03839
03840
03841
03842
03843
03844
03845
03846
03847
03848
03849
03850
03851
03852
03853
03854
03855
03856
03857
03858
03859
03860
03861
03862
03863
03864
03865
03866
03867
03868
03869
03870
03871
03872
03873
03874
03875
03876
03877
03878
03879
03880
03881
03882
03883
03884
03885
03886
03887
03888
03889
03890
03891
03892
03893
03894
03895
03896
03897
03898
03899
03900
03901
03902
03903
03904
03905
03906
03907
03908
03909
03910
03911
03912
03913
03914
03915
03916
03917
03918
03919
03920
03921
03922
03923
03924
03925
03926
03927
03928
03929
03930
03931
03932
03933
03934
03935
03936
03937
03938
03939
03940
03941
03942
03943
03944
03945
03946
03947
03948
03949
03950
03951
03952
03953
03954
03955
03956
03957
03958
03959
03960
0396

In [28]:
submission.to_csv("submission1.csv", index=False)

In [29]:
# kaggle API configuration
import os
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

In [30]:
!kaggle competitions submit -c super-ai-ss-5-named-entity-recognition -f submission1.csv -m "train_eval wangchan"

100%|██████████████████████████████████████| 2.41M/2.41M [00:00<00:00, 5.32MB/s]
Successfully submitted to NLP: Named Entity Recognition